In [11]:
!pip install clingo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.4 MB/s eta 0:00:00


In [12]:
import clingo

In [13]:
### DO NOT CHANGE THIS CELL

%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt

import clingo
from IPython.display import clear_output

In [14]:
### YOU MAY CHANGE THIS CELL TO TEST YOUR SOLUTION ON DIFFERENT INPUTS

nodes = list(range(1, 6+1))
edges = [
    (1,2),
    (2,3),
    (3,4),
    (4,5),
    (2,6),
    (3,6),
]
factories = [
    {
        'location': 1,
        'load': 10,
    },
]
delivery_points = [
    {
        'location': 4,
        'demand': 2,
    },
    {
        'location': 5,
        'demand': 2,
    },
]
charging_points = [
    {
        'location': 2,
    },
    {
        'location': 6,
    },
]
trucks = [
    {
        'location': 1,
        'load': 0,
        'capacity': 1,
        'charge level': 2,
        'charge capacity': 10,
    },
    {
        'location': 2,
        'load': 0,
        'capacity': 1,
        'charge level': 2,
        'charge capacity': 10,
    },
]

num_time_steps = 40

In [258]:
### DO NOT CHANGE THIS CELL

warehouse_locations = [
    warehouse_info['location']
    for warehouse_info in factories
]
truck_locations = [
    truck_info['location']
    for truck_info in trucks
]
delivery_locations = [
    dp_info['location']
    for dp_info in delivery_points
]
charging_locations = [
    cp_info['location']
    for cp_info in charging_points
]

class InvalidInputError(Exception):
    pass

intersection = [location for location in warehouse_locations if location in delivery_locations]
if len(intersection) > 0:
    raise InvalidInputError(f'node {intersection[0]} is both a warehouse and a delivery location')

intersection = [location for location in warehouse_locations if location in charging_locations]
if len(intersection) > 0:
    raise InvalidInputError(f'node {intersection[0]} is both a warehouse and a charging location')

intersection = [location for location in delivery_locations if location in charging_locations]
if len(intersection) > 0:
    raise InvalidInputError(f'node {intersection[0]} is both a delivery and a charging location')

In [307]:
# Load example inputs 
asp_program = ""
asp_program += f"     #const t={num_time_steps}.\n"
asp_program += f"     #const tr={len(trucks)}.\n"
asp_program += f"     #const num_nodes={len(nodes)}.\n"
#asp_program += f"     #const charge_cap={trucks[0]['charge capacity']}.\n" #all trucks have the same battery capacity safe to make that assumption??
for node in nodes:
  asp_program += f"     node({node}).\n"

for edge in edges:
  asp_program += f"     edge({edge[0]},{edge[1]}).\n"

max_factory_load = 0
for i, factory in enumerate(factories):
  location = factory['location']
  load = factory['load']
  if load >max_factory_load:
    max_factory_load = load
  asp_program += f"     factory_loc({location}).\n"
  asp_program += f"     factory_supply({location},{load}).\n"

asp_program += f"     #const loads_f={max_factory_load}.\n"

max_delivery_load = 0
for d_point in delivery_points:
  location = d_point['location']
  demand = d_point['demand']
  if demand >max_delivery_load:
    max_delivery_load = demand
  asp_program += f"     delivery_point({location}).\n"
  asp_program += f"     delivery_asked({location},{demand}).\n"

asp_program += f"     #const loads_d={max_delivery_load }.\n"

for c_point in charging_points:
  location = c_point['location']
  asp_program += f"     charging_point({location}).\n"

max_load_for_truck = 0
for i, truck in enumerate(trucks):
  location = truck['location']
  load = truck['load']
  load_capacity = truck['capacity']
  if load_capacity > max_load_for_truck:
    max_load_for_truck = load_capacity
  current_level = truck['charge level']
  c_capacity = truck['charge capacity']
  asp_program += f"     truck({i+1}, {load_capacity}, {c_capacity}).\n"
  asp_program += f"     truck_at({i+1},{location}).\n"
  asp_program += f"     cargo({i+1},{load}).\n"
  asp_program += f"     battery({i+1}, {current_level}).\n"
  #asp_program += f"     truck_max_charge({i+1}, {c_capacity}).\n"

asp_program += f"     #const loads_t={max_load_for_truck}.\n"

In [308]:
asp_program += """  
    %charge_level(0..charge_cap).
    %load_for_trucks(0..loads_t).
    load_for_f(0..loads_f).
    load_for_d(0..loads_d).

    % Generate times till maximum number of time steps
    possible_time(1..t).
    { time(T) : possible_time(T) }.
    :- not time(1). %first time step is time 1
    :- possible_time(T), possible_time(T+1), not time(T), time(T+1). %times are sequential


    % Max time is last time step
    %max_time(MaxTime) :- MaxTime = #max { T : time(T) }.
    last_timestep(M) :- time(M), not time(T) : possible_time(T), T > M.

    %time(1..t).

    % Edges are undirected
    edge(X,Y) :- edge(Y,X).

    % These are the possible actions that we can take
    action(wait).
    action(move(X,Y)) :- edge(X,Y).
    action(load).
    action(unload).
    action(charge).

    % We perform exactly one action at each time step
    1 { do(T, Tr, A) : action(A) } 1 :- time(T), T<M, last_timestep(M),  truck(Tr,_,_).
    
    """

In [309]:
#NEW WAY
asp_program += """

    %initial state
    state(1, truck_at(Tr, Loc)) :- truck_at(Tr, Loc) .
    state(1, battery(Tr, L)) :- battery(Tr, L).
    state(1, cargo(Tr, P)) :- cargo(Tr, P).
    state(1, delivery_asked(N, P)):- delivery_asked(N,P).
    state(1, factory_supply(N, P)) :- factory_supply(N,P).

    
    % Create what will be treated as fluent
    fluent(truck_at(Tr, Loc)) :- truck(Tr,_,_), node(Loc). 
    fluent(battery(Tr, 0..C)) :- truck(Tr, _, C). 
    fluent(cargo(Tr, 0..L)) :- truck(Tr, L, _).
    fluent(factory_supply(F, L)) :- factory_loc(F), load_for_f(L).     %state(1, factory_supply(F, L)).
    fluent(delivery_asked(Loc, L)) :- delivery_point(Loc), load_for_d(L).                               %state(1, delivery_asked(Loc, L)).

  
    % 1. Fluent states stay true unless explicitly made false
    state(T,S) :- time(T), fluent(S), state(T-1,S), not make_false(T,S).
    % 2. Fluent states that are explicitly made true, become true
    state(T,S) :- time(T), fluent(S), make_true(T,S).

    %move
    make_true(T+1, truck_at(Tr, Y)) :- do(T, Tr, move(X,Y)).                               %location(T, Tr, X), node(Y), time(T).
    make_false(T+1, truck_at(Tr, X)) :- do(T, Tr, move(X,Y)).

    make_true(T+1, battery(Tr, B-1)) :- do(T, Tr, move(_,_)), state(T, battery(Tr, B)).      %, time(T+1).
    make_false(T+1, battery(Tr, B)) :- do(T, Tr, move(_,_)), state(T, battery(Tr, B)).       %, time(T+1).

    %load
    make_true(T+1, cargo(Tr, L+1)) :- do(T, Tr, load), state(T,cargo(Tr, L)).             % time(T+1). %location(T, Tr, X), node(Y), time(T).
    make_false(T+1, cargo(Tr, L)) :- do(T, Tr, load),  state(T,cargo(Tr, L)).             %time(T+1),

    make_true(T+1, factory_supply(F, L-1)) :- do(T, Tr, load), state(T, factory_supply(F, L)), state(T, truck_at(Tr, F)).     %, time(T+1).
    make_false(T+1, factory_supply(F, L)) :- do(T, Tr, load), state(T, factory_supply(F, L)), state(T, truck_at(Tr, F)).      %time(T+1).

    %unload
    make_true(T+1, cargo(Tr, L-1)) :- do(T, Tr, unload), state(T,cargo(Tr, L)).       % time(T+1). %location(T, Tr, X), node(Y), time(T).
    make_false(T+1, cargo(Tr, L)) :- do(T, Tr, unload), state(T,cargo(Tr, L)).

    make_true(T+1, delivery_asked(X, L-1)) :- do(T, Tr, unload), state(T, delivery_asked(X, L)), state(T, truck_at(Tr, X)).   %, time(T+1).
    make_false(T+1, delivery_asked(X, L)) :- do(T, Tr, unload), state(T, delivery_asked(X, L)), state(T, truck_at(Tr, X)).    %, time(T+1).

    %charge
    make_true(T+1, battery(Tr, B+1)) :- do(T, Tr, charge), state(T, battery(Tr, B)).   %, time(T+1).
    make_false(T+1, battery(Tr, B)) :- do(T, Tr, charge), state(T, battery(Tr, B)).    %, time(T+1).   
    
  
"""

In [310]:
asp_program += """
    % truck loading
    % Cannot load if not at factory
    :- do(T, Tr, load), state(T, truck_at(Tr, Loc)), not factory_loc(Loc). 
    % Cannot load if the factory is empty
    :- do(T, Tr, load), state(T, truck_at(Tr, Loc)), state(T, factory_supply(Loc, S)), S<1.
    % Cannot load when load capacity is full
    :- do(T, Tr, load), state(T, cargo(Tr,L)), truck(Tr,LC,_), L>=LC.

    % truck unloading
    % Cannot unload if not at delivery point
    :- do(T, Tr, unload), state(T, truck_at(Tr, Loc)), not delivery_point(Loc).
    % Cannot unload when truck load is 0
    :- do(T, Tr, unload),  state(T, cargo(Tr, L)), L<1.
    % Cannot unload if the demand is 0
    :- do(T, Tr, unload), state(T, truck_at(Tr, Loc)), state(T, delivery_asked(Loc, D)), D<1.

    %charging
    % Cannot charge if truck is not in charging location
    :- do(T, Tr, charge),  state(T, truck_at(Tr, Loc)), not charging_point(Loc).
    % Cannot charge when battery is full
    :- do(T,Tr, charge), state(T, battery(Tr, B)), truck(Tr,_,C), B>=C.

    %moving
    % Cannot move from node where the truck is not at
    :- do(T, Tr, move(X,Y)), state(T, truck_at(Tr, Z)), Z!=X.
    % Cannot move when battery is empty
    :- do(T, Tr, move(X,Y)), state(T, battery(Tr, B)), B<1.
    % At each location, there can be at most one truck at the same time.
    :- do(T, Tr1, move(X,Y)), state(T+1, truck_at(Tr2, Y)), Tr1!=Tr2.
    % Trucks cannot pass each other on a piece of road—e.g., trucks at adjacent locations cannot swap places in a single time step.
    :- do(T, Tr1, move(X1,Y1)), do(T, Tr2, move(X2,Y2)), Tr1 != Tr2, X1=Y2, Y1=X2.
    

    

    %* 
    Cannot move when battery is empty
    %:- do(T, Tr, move(_,_)), battery(T, Tr, 0), time(T).
    :- do(T, Tr, move(_)), state(T, battery(Tr, 0)).

    % At each location, there can be at most one truck at the same time.
    :- state(T, truck_at(Tr1, Loc)), state(T, truck_at(Tr2, Loc)), Tr1!=Tr2.

    % Trucks cannot pass each other on a piece of road—e.g., trucks at adjacent locations cannot swap places in a single time step.
    :- do(T, Tr1, move(X)), do(T, Tr2, move(Y)), state(T,truck_at(Tr1, X)), state(T, truck_at(Tr2,Y)), Tr1 != Tr2.

    % Cannot move to node without an edge
    :- do(T, Tr, move(Y)), state(T, truck_at(Tr, X)), not edge(X,Y).

    
    % Cannot charge if truck is not in charging location
    :- do(T, Tr, charge),  state(T, truck_at(Tr, Loc)), not charging_point(Loc).

    % Cannot charge when battery is full
    :- do(T,Tr, charge), state(T, battery(Tr, C)), truck(T,_,C).

     % Cannot unload if not at delivery point
    :- do(T, Tr, unload), state(T, truck_at(Tr, Loc)), not delivery_point(Loc).

    % Cannot unload if the demand is 0
    :- do(T, Tr, unload), state(T, truck_at(Tr, Loc)), state(T, delivery_asked(Loc, 0)).

    % Cannot unload when truck load is 0
    :- do(T, Tr, unload),  state(T, cargo(Tr, 0)).

    % Cannot load if not at factory
    :- do(T, Tr, load), state(T, truck_at(Tr, Loc)), not factory_loc(Loc).

    % Cannot load if the factory is empty
    :- do(T, Tr, load), state(T, truck_at(Tr, Loc)), state(T, factory_supply(Loc, S)), S<1.

    % Cannot load when load capacity is full
    :- do(T, Tr, load), state(T, cargo(Tr,LC)), truck(Tr,LC,_).

    % A truck cannot be left without enough battery to reach a charging point
    %:- truck_at(Tr, X), battery(Tr, L), charging_point_dist(X, DP), L<DP.
    *%


    
      
"""

In [311]:
#changes of the last cell according to new way
asp_program += """
    all_distances(0..MD) :- MD = #max { N : node(N) }.
    distance(X,X,0) :- node(X).
    distance(X,Y,1) :- edge(X,Y).
    distance(X, Y, D1+1) :- all_distances(D1+1), distance(X, Z, D1), edge(Z,Y), not distance(X,Y, D2) : all_distances(D2), D1>D2.

    charging_point_dist(X, PD) :- node(X), PD = #min{ D : distance(X,Y,D), charging_point(Y)}.

    can_charge(T, Tr) :- state(T, battery(Tr, B)), state(T, truck_at(Tr, X)), charging_point_dist(X, D), B>=D.
    :- not can_charge(T, Tr), truck(Tr,_,_), time(T).

    truck_parked(T, Tr) :- state(T, truck_at(Tr, Loc)), charging_point(Loc).
    all_parked(T) :- truck_parked(T,Tr) : truck(Tr,_,_); time(T).

    %deliveries_done(T, Loc) :- state(T, delivery_asked(Loc,0)).
    
    %truck_empty(T, Tr) :- state(T, cargo(Tr,0)).
    %find_ch_point(T, Tr) :- state(T, truck_at(Tr, Loc)), state(T, battery(Tr, L)), charging_point_dist(Loc, D), D<=L.

    goal_reached(T) :- time(T),
          state(T, delivery_asked(Loc, 0)): delivery_point(Loc);
          all_parked(T),
          state(T, cargo(Tr, 0)) : truck(Tr,_,_).


	  %*deliveries_done(T, Loc): delivery_point(Loc);
        truck_parked(T, Tr): truck(Tr,_,_);
        truck_empty(T, Tr): truck(Tr,_,_) ;
        find_ch_point(T, Tr): truck(Tr,_,_).
    *%
    :- not goal_reached(T), last_timestep(T).
    %:- time(M), goal_reached(t), M>t.
    :- last_timestep(M), goal_reached(T), M>T.
   
    #minimize { T : last_timestep(T)}.
    %#minimize { 1,T : time(T) }.
    %#minimize { 1,T : do(T, _, wait) }.
    
       

"""

In [ ]:
print(asp_program)

In [280]:
### DO NOT CHANGE THIS CELL

# Some machinery to (dynamically) print things in the notebook
log_str = ""
def reset_log():
    global log_str
    log_str = ""
def log(msg):
    global log_str
    print(msg, flush=True)
    log_str += f"{msg}\n"
def print_log():
    global log_str
    print(log_str, end='', flush=True)
    
def display_solution(model):
    """
    Displays a solution specified by the answer set
    in a human-readable format.
    
    Assumes that the answer set program uses do/3
    to specify actions, and uses truck_at/2 to
    specify the initial locations of the trucks.
    """
    actions = {}
    truck_locations = {}
    for atom in model.symbols(atoms=True):
        if atom.name == "do":
            time = atom.arguments[0].number
            truck = str(atom.arguments[1])
            action = str(atom.arguments[2])
            print(time, truck, action)
            if time not in actions:
                actions[time] = {}
            actions[time][truck] = action
        elif atom.name == "truck_at":
            truck_locations[str(atom.arguments[0])] = str(atom.arguments[1])
    solution_str = "\n"
    solution_str += " "*5 + "".join([
        f"{truck:<5} @ {truck_locations[truck]:<10}"
        for truck in sorted(truck_locations)
    ]) + "\n"
    solution_str += "-"*(5+18*len(truck_locations)) + "\n"
    for time in sorted(actions):
        time_str = f"{time}:"
        solution_str += f"{time_str:<4} " + "".join([
            f"{actions[time][truck]:<18}"
            for truck in sorted(truck_locations)
        ]) + "\n"
    return solution_str

def find_and_output_solution(
    asp_program,
    timeout=10,
    num_threads=1,
):
    """
    Calls the ASP solver to find answer sets,
    and translates these to solutions,
    which are printed in human-readable format.
    """

    reset_log()
    log("..Grounding..")
    
    control = clingo.Control([f"--parallel-mode={num_threads}"])
    control.add("base", [], asp_program)
    control.ground([("base", [])])

    control.configuration.solve.models = 1
    control.configuration.solve.opt_mode = "optN"
    
    def interpret_model(model):
        clear_output(wait=True)
        print_log()
        sol = display_solution(model)
        print(sol)

    if timeout > 0:
        log(f"..Solving with timeout {timeout}s..")
        with control.solve(
            async_=True,
            on_model=lambda model: interpret_model(model),
        ) as handle:
            finished = handle.wait(timeout)
            if not finished:
                log("..Stopped after timeout..")
            else:
                log("..Stopped..")
                time = control.statistics['summary']['times']['total']
                log(f"..Total solving time: {time:.2f} seconds..")
    else:
        log("..Solving..")
        control.solve(
            on_model=lambda model: interpret_model(model),
        )
        log("..Stopped..")
        time = control.statistics['summary']['times']['total']
        log(f"..Total solving time: {time:.2f} seconds..")

In [312]:
find_and_output_solution(
    asp_program,
    timeout=0,
    num_threads=4,
)

..Grounding..
..Solving..
1 1 load
2 1 move(1,2)
3 1 charge
4 1 charge
5 1 charge
6 1 charge
7 1 charge
8 1 charge
9 1 charge
10 1 charge
11 1 charge
12 1 move(2,3)
13 1 move(3,4)
14 1 move(4,5)
15 1 unload
16 1 move(5,4)
17 1 move(4,3)
18 1 move(3,2)
19 1 charge
20 1 move(2,1)
21 1 load
22 1 move(1,2)
23 1 charge
24 1 charge
25 1 move(2,6)
26 1 charge
27 1 charge
28 1 move(6,3)
29 1 move(3,4)
30 1 move(4,5)
31 1 unload
32 1 move(5,4)
33 1 move(4,3)
34 1 move(3,2)
1 2 move(2,6)
2 2 charge
3 2 charge
4 2 charge
5 2 charge
6 2 charge
7 2 charge
8 2 charge
9 2 wait
10 2 charge
11 2 charge
12 2 move(6,2)
13 2 move(2,1)
14 2 load
15 2 move(1,2)
16 2 charge
17 2 charge
18 2 move(2,6)
19 2 charge
20 2 charge
21 2 move(6,3)
22 2 move(3,4)
23 2 unload
24 2 move(4,3)
25 2 move(3,2)
26 2 move(2,1)
27 2 load
28 2 move(1,2)
29 2 move(2,3)
30 2 move(3,4)
31 2 unload
32 2 move(4,3)
33 2 move(3,6)
34 2 wait

     1     @ 1         2     @ 2         
-----------------------------------------
1:   load 